In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

os.chdir(os.getenv("PROJECT_ROOT_DIR"))
print(os.getcwd())

/home/truonghm/learning/m1-internship/malicious-code-detection


In [2]:
import pandas as pd

In [17]:
url_df = pd.read_csv("data/malicious_phish.csv", usecols=[1,2,3], header=0, names=["url", "type", "super_type"])

In [18]:
url_df.head()

,url,type,super_type
0,http://br-icloud.com.br,phishing,badjs
1,http://mp3raid.com/music/krizz_kaliko.html,benign,goodjs
2,http://bopsecrets.org/rexroth/cr/1.htm,benign,goodjs
3,http://www.garage-pirenne.be/index.php?option=...,defacement,badjs
4,http://adventure-nicaragua.net/index.php?optio...,defacement,badjs


In [19]:
url_df = url_df.drop_duplicates(subset=['url'])

In [20]:
url_df["super_type"] = url_df["type"].apply(lambda x: "badjs" if x != "benign" else "goodjs")

In [21]:
url_df.reset_index(drop=False, inplace=True)

In [22]:
url_df.head()

,index,url,type,super_type
0,0,http://br-icloud.com.br,phishing,badjs
1,1,http://mp3raid.com/music/krizz_kaliko.html,benign,goodjs
2,2,http://bopsecrets.org/rexroth/cr/1.htm,benign,goodjs
3,3,http://www.garage-pirenne.be/index.php?option=...,defacement,badjs
4,4,http://adventure-nicaragua.net/index.php?optio...,defacement,badjs


In [13]:
url_df.to_csv("data/malicious_phish.csv", index=False)

In [14]:
url_df.head()

,Unnamed: 0,url,type,super_type
0,0,http://br-icloud.com.br,phishing,badjs
1,1,http://mp3raid.com/music/krizz_kaliko.html,benign,goodjs
2,2,http://bopsecrets.org/rexroth/cr/1.htm,benign,goodjs
3,3,http://www.garage-pirenne.be/index.php?option=...,defacement,badjs
4,4,http://adventure-nicaragua.net/index.php?optio...,defacement,badjs


In [39]:
url_df.groupby("type").count()

,url,super_type,valid
type,,,
benign,428103,428103,4
defacement,96457,96457,3
malware,32520,32520,0
phishing,94111,94111,1


In [13]:
# set the super type: if type is not benign, set it to malicious

url_df["super_type"] = url_df["type"].apply(lambda x: "malicious" if x != "benign" else "benign")
url_df["valid"] = None

In [40]:
url_df.shape

(651191, 4)

In [11]:
import json
import os

import bs4
import pymongo
import requests

In [27]:
url_df.head(10)

,url,type,super_type,valid
0,http://br-icloud.com.br,phishing,malicious,False
1,http://mp3raid.com/music/krizz_kaliko.html,benign,benign,False
2,http://bopsecrets.org/rexroth/cr/1.htm,benign,benign,False
3,http://www.garage-pirenne.be/index.php?option=...,defacement,malicious,False
4,http://adventure-nicaragua.net/index.php?optio...,defacement,malicious,False
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,benign,False
6,http://espn.go.com/nba/player/_/id/3457/brando...,benign,benign,False
7,http://yourbittorrent.com/?q=anthony-hamilton-...,benign,benign,None
8,http://www.pashminaonline.com/pure-pashminas,defacement,malicious,False
9,http://allmusic.com/album/crazy-from-the-heat-...,benign,benign,None


In [29]:
import concurrent.futures

In [42]:
skip = 100
limit = 10000
url_df.iloc[skip : skip + limit].index

RangeIndex(start=100, stop=10100, step=1)

In [35]:

def extract_js(url):
    try:
        response = requests.get(url, timeout=3)
        # print("Status code:", response.status_code)
        if response.status_code != 200 or not response.text:
            return ""
        soup = bs4.BeautifulSoup(response.text, "html.parser")
        scripts = soup.find_all("script")
        js_code = ""
        for script in scripts:
            if script.contents:
                js_code += script.contents[0]
        return js_code
    except requests.exceptions.RequestException as e:
        raise


def extract_js_and_save(url, label, super_label, is_valid):
    if is_valid is False:
        return None

    if not url.startswith("http") and not url.startswith("https"):
        new_url = "http://" + url
    else:
        new_url = url

    try:
        js_code = extract_js(new_url)
    except Exception as e:
        print(f"Failed ({super_label}): {new_url}, {str(e)}")
        return None

    if js_code == "":
        return None

    return {
        "url": new_url,
        "label": label,
        "super_label": super_label,
        "js_code": js_code
    }
    print(f"Success ({super_label}): {new_url}")

result = []
n_url = 10

# use a thread pool to execute the tasks
with concurrent.futures.ThreadPoolExecutor() as executor:
    # prepare the arguments for each task
    args = [(url, label, super_label, is_valid) for url, label, super_label, is_valid in url_df[url_df["valid"].isnull()].head(n_url).itertuples(index=False)]
    # submit the tasks and wait for the results
    for res in executor.map(lambda x: extract_js_and_save(*x), args):
        if res:
            result.append(res)

url_df.to_csv("data/archive/malicious_phish.csv", index=False)

Failed (benign): http://quickfacts.census.gov/qfd/maps/iowa_map.html, HTTPConnectionPool(host='quickfacts.census.gov', port=80): Max retries exceeded with url: /qfd/maps/iowa_map.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f56f46b8b80>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
Failed (malicious): http://larcadelcarnevale.com/catalogo/palloncini, HTTPConnectionPool(host='larcadelcarnevale.com', port=80): Max retries exceeded with url: /catalogo/palloncini (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f56f46b8040>, 'Connection to larcadelcarnevale.com timed out. (connect timeout=3)'))
Failed (malicious): http://www.lebensmittel-ueberwachung.de/index.php/aktuelles.1, HTTPConnectionPool(host='www.lebensmittel-ueberwachung.de', port=80): Max retries exceeded with url: /index.php/aktuelles.1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f56f